In [413]:
import pickle
import numpy as np
import pandas as pd

In [414]:
n = 30

data = {
    'user_id': [f'U{i}' for i in range(1, n + 1)],
    'age': np.random.randint(18, 36, size=n),
    'gender': np.random.choice(['M', 'F', 'Other'], size=n),
    'past_engagement_score': np.round(np.random.uniform(0.2, 0.98, size=n), 2)
}

# Create the DataFrame
df = pd.DataFrame(data)


In [415]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                30 non-null     object 
 1   age                    30 non-null     int32  
 2   gender                 30 non-null     object 
 3   past_engagement_score  30 non-null     float64
dtypes: float64(1), int32(1), object(2)
memory usage: 972.0+ bytes


In [416]:
df.head()

,user_id,age,gender,past_engagement_score
0,U1,34,F,0.41
1,U2,21,Other,0.66
2,U3,29,F,0.95
3,U4,26,M,0.58
4,U5,34,F,0.36


In [417]:
df['gender']=np.where(df['gender'] == 'M',1,
                np.where(df['gender'] == 'F',0,-1))

In [418]:
gb = pd.read_csv('Interests_list.csv')
gb['key'] = 1
df['key'] = 1

df = pd.merge(df, gb, how='inner', on='key')
df = df.drop(['key', 'Interest'], axis=1)
df.shape

(300, 5)

In [419]:
import joblib
model = "3-Model RF 2025-09-30.pkl"
print(model)
model = joblib.load(model)

3-Model RF 2025-09-30.pkl


In [420]:
x = df[['past_engagement_score','age','ID']]
print(x.shape)


df['Engagement']=model.predict_proba(x)[:, 1]
df = df.sort_values(['user_id','Engagement','ID'], ascending=[False,False,False]).reset_index(drop=True)
print(df['ID'].unique())
print(df.shape)






(300, 3)
[0 2 1 9 8 7 3 6 4 5]
(300, 6)


In [421]:
gb = pd.read_csv('Interests_list.csv')
df = pd.merge(df, gb, how='left', on='ID')


df = df.sort_values(['user_id','Engagement'], ascending=[True,False]).reset_index(drop=True)
df=df.drop(['ID'],axis=1)


In [422]:
df = df[df['user_id']!=df['user_id'].shift(3)]
df.shape

(90, 6)

In [423]:
df

,user_id,age,gender,past_engagement_score,Engagement,Interest
0,U1,34,0,0.41,0.504041,literature
1,U1,34,0,0.41,0.504041,fitness
2,U1,34,0,0.41,0.476457,tech
10,U10,21,0,0.78,0.845230,gaming
11,U10,21,0,0.78,0.832398,sports
...,...,...,...,...,...,...
281,U8,30,1,0.86,0.917446,travel
282,U8,30,1,0.86,0.916726,gaming
290,U9,27,-1,0.27,0.260245,sports
291,U9,27,-1,0.27,0.231490,gaming


In [ ]:
# Grouped by User_id and joined values with ','
Output = df.groupby('user_id', as_index=False)['Interest'].apply(lambda x: ','.join(x)).rename(columns={'Interest': 'top_3_interests'})

In [425]:

Output.to_csv('4-Output.csv', index=False)
Output.shape

(30, 2)